In [3]:
import sys
sys.path.append('..')
import dataset
import logging
import mxnet as mx
from mxnet import autograd
from mxnet import gluon, image, nd, init, metric
import model
import utils
from dataset import load_data_ChestX_ray14
from mxnet.gluon import nn
import matplotlib.pyplot as plt

C:\Users\weij\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\weij\anaconda3\lib\site-packages\h5py\tests\old\test_attrs_data.py:251: DeprecationWarning: invalid escape sequence \H
  s = b"Hello\x00\Hello"
C:\Users\weij\anaconda3\lib\site-packages\sklearn\externals\joblib\func_inspect.py:53: DeprecationWarning: invalid escape sequence \<
  '\<doctest (.*\.rst)\[(.*)\]\>', source_file).groups()
C:\Users\weij\anaconda3\lib\site-packages\sklearn\externals\joblib\_memory_helpers.py:10: DeprecationWarning: invalid escape sequence \s
  cookie_re = re.compile("coding[:=]\s*([-\w.]+)")
C:\Users\weij\anaconda3\lib\site-packages\sklearn\metrics\classification.py:349: DeprecationWarning: invalid escape sequence \k
  """
C:\Users\weij\anaconda3\lib

In [7]:
lr = 0.01
num_epoches = 100
momentum = 0.5
ctx = utils.try_all_gpus()

In [12]:
train_iter, test_iter = load_data_ChestX_ray14(32, resize=224)

utils.set_logging_level(level=logging.WARNING)

In [13]:
net = model.get_pretrained_resnet50(ctx)

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': momentum})
# loss_fn = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=False)
loss_fn = gluon.loss.SigmoidBinaryCrossEntropyLoss()

In [15]:
utils.evaluate_AUC(test_iter, net, ctx)
# utils.evaluate_loss(test_iter, net, ctx)

array([0.5725189 , 0.50252365, 0.55912434, 0.52862209, 0.48606037,
       0.5242294 , 0.48661877, 0.4940632 , 0.55028311, 0.46767346,
       0.52611852, 0.42321167, 0.45524648, 0.68381255])

In [ ]:
utils.train_parallel(train_iter, test_iter, net, loss_fn, trainer, ctx, num_epochs=num_epoches )